# Sea surface temperature Canary

Plot the SST in the Canary Current Upwelling System.

In [10]:
import sys
sys.path.insert(0, '..')
import os
import glob
from filament import filament
import datetime
import netCDF4
import cmocean
import logging
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cartopyticker
reload(filament)
plt.style.use("filament.mplstyle")

In [5]:
logger = logging.getLogger("filamentPlot")
logger.setLevel(logging.INFO)
logging.info("Starting")
plotvis = 0

In [14]:
regionname= "BloomJune2024"
datadir = f"/home/ctroupin/data/SST/{regionname}/"

figdir = f"../figures/SST/{regionname}/"
visibledir = "/data/Visible/"
datafilelist = sorted(glob.glob(os.path.join(datadir, "*SST*.nc")))

nfiles = len(datafilelist)

logger.info("Processing {} files".format(nfiles))
    
domain = filament.regions["canarydomain"]
myproj = ccrs.Mercator(central_longitude=0.5 * (domain[0] + domain[1]), min_latitude=domain[2], 
                       max_latitude=domain[3])

INFO:filamentPlot:Processing 26 files


In [15]:
if not(os.path.exists(figdir)):
    os.makedirs(figdir)

## Loop on files

In [20]:
for ifile, datafile in enumerate(datafilelist[2:3]):
    logger.info("Working on {}".format(datafile))
    
    #chloro = filament.Chloro()=
    #chloro.read_from_oceancolorL2(datafile)
    
    
    with netCDF4.Dataset(datafile) as nc:
        proclevel = nc.processing_level
    
    sst = filament.SST()
    
    # Select the reader according to processing level
    if proclevel == "L3 Mapped":
        logger.info("Level-3 data")
        sst.read_from_oceancolorL3(datafile, coordinates)
    elif proclevel == "L2":
        logger.info("Level-2 data")
        sst.read_from_oceancolorL2(datafile)
        sst.field = np.ma.masked_where(sst.qflag > 2, sst.field)    
    titletext = sst.get_title()
    

    sstmin, sstmax = 17.5, 22.5
    
    figname = os.path.join(figdir, sst.get_figname())

    fig = plt.figure(figsize=(12, 10))
    ax = plt.subplot(111, projection=myproj)
    ax.set_extent(domain)
    ax.set_title(titletext, fontsize=24)
    
    
    iii = 0
    #for tlon, tlat in zip(tracklon, tracklat):
    #    iii += 1
    #    ax.text(tlon, tlat, iii, horizontalalignment='center', verticalalignment='center',
    #        fontsize=15)
        
    #vis.add_to_plot(ax, myproj)
    #sst.field[sst.lat.data == -999.] = np.nan
    #sst.field[sst.lon.data == -999.] = np.nan
    #sst.lat[sst.lat.data == -999.] = np.nan
    #sst.lon[sst.lon.data == -999.] = np.nan
    
    #sst.lat = np.ma.masked_equal(sst.lat, -999.0, copy=True)
    #sst.lon = np.ma.masked_equal(sst.lon, -999.0, copy=True)
    pcm = ax.pcolormesh(sst.lon.data, sst.lat.data, sst.field, cmap=plt.cm.RdYlBu_r, 
                        vmin=sstmin, vmax=sstmax, transform=filament.datacrs)
    
    cb = plt.colorbar(pcm, extend="both", shrink=0.75)
    cb.set_label("°C", rotation=0, ha="left")

    iii = 0        
    #vis.add_to_plot(ax, myproj)

    ax.add_feature(filament.coast_f, linewidth=.2, color=".5")
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=.5, color='.25', linestyle='--', zorder=2)
    gl.top_labels = False
    gl.right_labels = False

    plt.savefig(figname)
    # plt.show()
    plt.close(fig)

INFO:filamentPlot:Working on /home/ctroupin/data/SST/BloomJune2024/AQUA_MODIS.20240624T041501.L2.SST4.NRT.nc
INFO:filamentPlot:Level-2 data
INFO:filamentPlot:Working on /home/ctroupin/data/SST/BloomJune2024/AQUA_MODIS.20240624T151001.L2.SST.NRT.nc
INFO:filamentPlot:Level-2 data
INFO:filamentPlot:Working on /home/ctroupin/data/SST/BloomJune2024/AQUA_MODIS.20240625T031501.L2.SST4.NRT.nc
INFO:filamentPlot:Level-2 data
INFO:filamentPlot:Working on /home/ctroupin/data/SST/BloomJune2024/AQUA_MODIS.20240625T141501.L2.SST.NRT.nc
INFO:filamentPlot:Level-2 data
INFO:filamentPlot:Working on /home/ctroupin/data/SST/BloomJune2024/AQUA_MODIS.20240626T145500.L2.SST.NRT.nc
INFO:filamentPlot:Level-2 data
INFO:filamentPlot:Working on /home/ctroupin/data/SST/BloomJune2024/AQUA_MODIS.20240627T030001.L2.SST4.NRT.nc
INFO:filamentPlot:Level-2 data
INFO:filamentPlot:Working on /home/ctroupin/data/SST/BloomJune2024/AQUA_MODIS.20240627T140001.L2.SST.NRT.nc
INFO:filamentPlot:Level-2 data
INFO:filamentPlot:Workin